<center> <span style="font-size:30px;"><span style="color:black;"><b>SC1015 Mini Project</b></span></span> </center>
<br>
<center> <span style="font-size:30px;"><span style="color:black;"><b>Heart Disease Predictor</b></span></span> </center>


# Importing of Required Libraries

In [ ]:
!pip install plotly-express

In [ ]:
!pip list


In [ ]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics
import plotly_express as px

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score , ConfusionMatrixDisplay , classification_report, confusion_matrix
import warnings


# 1. Analyse of Data
For our mini-project we will be using the FramingHam data set from Kaggle. The aim of this project is to predict if a patient has a 10 years risk of future (CHD) coronary heart disease.
<br>
We will begin by importing the data and going through the variables in the data set

In [ ]:
heartData = pd.read_csv('framingham.csv')
heartData.head()

In [ ]:
# Information about the Variables
heartData.info()
heartData.shape

In [ ]:
heartData.describe()

Summary of results

# 2. Cleaning/Preprocessing of Data

## Missing Value Cleaning
Check to see if there is any null values in the data set

In [ ]:
# Percentage of null values in each column
(heartData.isnull().sum()/heartData.shape[0])*100

In [ ]:
heartData['education'].fillna(0,inplace=True)
heartData['cigsPerDay'].fillna(heartData['cigsPerDay'].where(heartData['currentSmoker']==1).median(),inplace=True)
heartData['BPMeds'].fillna(0,inplace=True)
heartData['totChol'].fillna(heartData['totChol'].median(),inplace=True)
heartData['BMI'].fillna(heartData['BMI'].median(),inplace=True)
heartData['heartRate'].fillna(heartData['heartRate'].where(heartData['currentSmoker']==1).median(),inplace=True)
heartData['glucose'].fillna(heartData['glucose'].where(heartData['diabetes']==0).median(),inplace=True)

In [ ]:
# Checking if there are any misisng values:
(heartData.isnull().sum()/heartData.shape[0])*100

## Outlier cleaning
Identify which variables needs outliers cleaning and remove the outlier rows

In [ ]:
# Backing up data sets for any just in case
heartData_Copy = heartData.copy()

In [ ]:
# Draw the distributions of all variables
f, axes = plt.subplots(16, 2, figsize=(20,75))

count = 0
for var in heartData:
    sb.boxplot(data = heartData[var], orient = "h", ax = axes[count,0])
    axes[count,0].set_title(f'Boxplot of {var}', color='DarkRed')  
    sb.violinplot(data = heartData[var], orient = "h", ax = axes[count,1])
    axes[count,1].set_title(f'Violinplot of {var}', color='DarkRed') 
    count += 1
    
#Calculate outliers
Q1 = heartData.quantile(0.25)
Q3 = heartData.quantile(0.75)
IQR = Q3 - Q1
print (((heartData < (Q1 - 1.5 * IQR)) | (heartData > (Q3 + 1.5 * IQR))).sum())

In [ ]:
# Extracting variables that requires outlier remove
outlier_variables = ['cigsPerDay','totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']

In [ ]:
# To remove outliers
for var in outlier_variables:
    Q1 = heartData[var].quantile(0.25)
    Q3 = heartData[var].quantile(0.75)
    IQR = Q3 - Q1
    UL = Q3 + 1.5 * IQR
    LL = Q1 - 1.5 * IQR
    heartData = heartData[(heartData[var] < UL) & (heartData[var] > LL)] 

In [ ]:
print('There were {} rows before outlier treatment.'.format(heartData_Copy.shape[0]))
print('There are {} rows after outlier treatment.'.format(heartData.shape[0]))
print('After outlier treatment number of rows lost are {}.'.format(heartData_Copy.shape[0] - heartData.shape[0]))

# 3. Exploratory Analysis
Now we have a processed data set, we will start by finding out which variables play a strong factor in determining the final prediction result.
<br>
First lets explore each variables by generating various graphs.


In [ ]:
heartData.describe()

In [ ]:
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        dataframe[feature].hist(bins=20,ax=ax,facecolor='midnightblue')
        ax.set_title(feature+" Distribution",color='DarkRed')
        
    fig.tight_layout()  
    plt.show()
draw_histograms(heartData,heartData.columns,6,3)

In [ ]:
heartData.TenYearCHD.value_counts()

In [ ]:
sb.countplot(x='TenYearCHD',data=heartData)

There are 3594 patients with no heart disease and 644 patients with risk of heart disease

We will generate a correlation matrix to evaluate the relationship between every variables in the data set by looking at its correlation coefficient.

In [ ]:
# Calculate the correlation matrix
correlation_matrix = heartData.corr()
# Plot the heatmap
plt.figure(figsize=(10, 8))
fig , ax = plt.subplots(figsize=(25 , 20))
sb.heatmap(correlation_matrix, annot=True, cmap='Greens', ax=ax)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Calculate percentage of males and females with TenYearCHD
male_chd_percentage = heartData[heartData['male'] == 1]['TenYearCHD'].mean() * 100
female_chd_percentage = heartData[heartData['male'] == 0]['TenYearCHD'].mean() * 100

# Plotting
plt.bar(['Male', 'Female'], [male_chd_percentage, female_chd_percentage], color=['blue', 'pink'])
plt.xlabel('Gender')
plt.ylabel('Percentage with TenYearCHD')
plt.title('Percentage of TenYearCHD by Gender')
plt.show()

In [ ]:
# Calculate percentage of individuals in different age groups with TenYearCHD
age_groups = [30, 40, 50, 60, 70]
age_chd_percentage = []

for age in age_groups:
    chd_percentage = heartData[(heartData['age'] >= age) & (heartData['age'] < age + 10)]['TenYearCHD'].mean() * 100
    age_chd_percentage.append(chd_percentage)

# Plotting
plt.bar(age_groups, age_chd_percentage, color='green')
plt.xlabel('Age')
plt.ylabel('Percentage with TenYearCHD')
plt.title('Percentage of TenYearCHD by Age Group')
plt.xticks(age_groups)
plt.show()

In [ ]:
# Calculate percentage of current smokers and non-smokers with TenYearCHD
current_smoker_chd_percentage = heartData[heartData['currentSmoker'] == 1]['TenYearCHD'].mean() * 100
non_smoker_chd_percentage = heartData[heartData['currentSmoker'] == 0]['TenYearCHD'].mean() * 100

# Plotting
plt.bar(['Current Smoker', 'Non-Smoker'], [current_smoker_chd_percentage, non_smoker_chd_percentage], color=['orange', 'purple'])
plt.xlabel('Smoking Status')
plt.ylabel('Percentage with TenYearCHD')
plt.title('Percentage of TenYearCHD by Smoking Status')
plt.show()

In [ ]:
# Calculate percentage of individuals in different cigarette per day groups with TenYearCHD
cigs_per_day_groups = [0, 10, 20, 30, 40, 50,60]
cigs_per_day_chd_percentage = []

for cigs_per_day in cigs_per_day_groups:
    chd_percentage = heartData[(heartData['cigsPerDay'] >= cigs_per_day) & (heartData['cigsPerDay'] < cigs_per_day + 10)]['TenYearCHD'].mean() * 100
    cigs_per_day_chd_percentage.append(chd_percentage)

# Plotting
plt.bar(cigs_per_day_groups, cigs_per_day_chd_percentage, color='red')
plt.xlabel('Cigarettes per Day')
plt.ylabel('Percentage with TenYearCHD')
plt.title('Percentage of TenYearCHD by Cigarettes per Day Group')
plt.xticks(cigs_per_day_groups)
plt.show()

In [ ]:
bmi_groups = [0, 18.5, 25, 30, 35, 40, float('inf')]
bmi_labels = ['Underweight', 'Normal', 'Overweight', 'Obese Class I', 'Obese Class II', 'Obese Class III']
bmi_chd_percentage = []

for i in range(len(bmi_groups) - 1):
    lower_bound = bmi_groups[i]
    upper_bound = bmi_groups[i + 1]
    chd_percentage = heartData[(heartData['BMI'] >= lower_bound) & (heartData['BMI'] < upper_bound)]['TenYearCHD'].mean() * 100
    bmi_chd_percentage.append(chd_percentage)

# Plotting
plt.bar(bmi_labels, bmi_chd_percentage, color='blue')
plt.xlabel('BMI Categories')
plt.ylabel('Percentage with TenYearCHD')
plt.title('Percentage of TenYearCHD by BMI Category')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
diabetes_chd_percentage = heartData[heartData['diabetes'] == 1]['TenYearCHD'].mean() * 100
no_diabetes_chd_percentage = heartData[heartData['diabetes'] == 0]['TenYearCHD'].mean() * 100

# Plotting
plt.bar(['Diabetes', 'No Diabetes'], [diabetes_chd_percentage, no_diabetes_chd_percentage], color=['orange', 'blue'])
plt.xlabel('Diabetes Status')
plt.ylabel('Percentage with TenYearCHD')
plt.title('Percentage of TenYearCHD by Diabetes Status')
plt.show()

In [ ]:
# Calculate percentage of individuals in different glucose level groups with TenYearCHD
glucose_groups = [0, 100, 125, 150, 200, float('inf')]
glucose_labels = ['Normal', 'Prediabetes', 'Borderline Diabetes', 'Diabetes', 'Severe Diabetes']
glucose_chd_percentage = []

for i in range(len(glucose_groups) - 1):
    lower_bound = glucose_groups[i]
    upper_bound = glucose_groups[i + 1]
    chd_percentage = heartData[(heartData['glucose'] >= lower_bound) & (heartData['glucose'] < upper_bound)]['TenYearCHD'].mean() * 100
    glucose_chd_percentage.append(chd_percentage)

# Plotting
plt.bar(glucose_labels, glucose_chd_percentage, color='green')
plt.xlabel('Glucose Level Categories')
plt.ylabel('Percentage with TenYearCHD')
plt.title('Percentage of TenYearCHD by Glucose Level Category')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
totChol_bins = [0, 200, 240, 280, float('inf')]
totChol_labels = ['Desirable (0-200)', 'Borderline High (200-240)', 'High (240-280)', 'Very High (>280)']

# Calculate percentage of TenYearCHD for each bin
totChol_chd_percentage = []
for i in range(len(totChol_bins) - 1):
    lower_bound = totChol_bins[i]
    upper_bound = totChol_bins[i + 1]
    chd_percentage = heartData[(heartData['totChol'] >= lower_bound) & (heartData['totChol'] < upper_bound)]['TenYearCHD'].mean() * 100
    totChol_chd_percentage.append(chd_percentage)

# Plotting
plt.bar(totChol_labels, totChol_chd_percentage, color='blue')
plt.xlabel('Total Cholesterol (mg/dL)')
plt.ylabel('Percentage with TenYearCHD')
plt.title('Percentage of TenYearCHD by Total Cholesterol Level')
plt.xticks(rotation=45)
plt.show()

In [ ]:
sysBP_bins = [0, 120, 130, 140, 180, float('inf')]
sysBP_labels = ['Normal (0-120)', 'Elevated (120-130)', 'Hypertension Stage 1 (130-140)', 'Hypertension Stage 2 (140-180)', 'Hypertensive Crisis (>180)']

# Calculate percentage of TenYearCHD for each bin of sysBP
sysBP_chd_percentage = []
for i in range(len(sysBP_bins) - 1):
    lower_bound = sysBP_bins[i]
    upper_bound = sysBP_bins[i + 1]
    chd_percentage = heartData[(heartData['sysBP'] >= lower_bound) & (heartData['sysBP'] < upper_bound)]['TenYearCHD'].mean() * 100
    sysBP_chd_percentage.append(chd_percentage)

# Plotting sysBP
plt.bar(sysBP_labels, sysBP_chd_percentage, color='red')
plt.xlabel('Systolic Blood Pressure (mmHg)')
plt.ylabel('Percentage with TenYearCHD')
plt.title('Percentage of TenYearCHD by Systolic Blood Pressure')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Define bins for Diastolic Blood Pressure (diaBP)
diaBP_bins = [0, 80, 90, 120, float('inf')]
diaBP_labels = ['Normal (0-80)', 'High-Normal (80-90)', 'Hypertension Stage 1 (90-120)', 'Hypertension Stage 2 (>120)']

# Calculate percentage of TenYearCHD for each bin of diaBP
diaBP_chd_percentage = []
for i in range(len(diaBP_bins) - 1):
    lower_bound = diaBP_bins[i]
    upper_bound = diaBP_bins[i + 1]
    chd_percentage = heartData[(heartData['diaBP'] >= lower_bound) & (heartData['diaBP'] < upper_bound)]['TenYearCHD'].mean() * 100
    diaBP_chd_percentage.append(chd_percentage)

# Plotting diaBP
plt.bar(diaBP_labels, diaBP_chd_percentage, color='green')
plt.xlabel('Diastolic Blood Pressure (mmHg)')
plt.ylabel('Percentage with TenYearCHD')
plt.title('Percentage of TenYearCHD by Diastolic Blood Pressure')
plt.xticks(rotation=45)
plt.show()

Summary of results

# 4. Exploring of Machine Learning Techniques
Now we explored the variables, we will be trying out different machine learning techniques to see which methods gives the best accuracy and using the confusion matrix to try out the prediction using test values.
<br>
We will be predicting TenYearCHD using the rest of variables. TenYearCHD is a boolean data type consisting of 0 and 1 (True and False), so machine learning techniques catering to classification will be explored.

In [ ]:
#Importing additional libraries for graph plotting of model results
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, ConfusionMatrixDisplay
from sklearn.model_selection import learning_curve

In [ ]:
X = heartData.drop('TenYearCHD' , axis= 'columns')
y = heartData['TenYearCHD']

Create appropriate datasets for Train and Test in an 80:20 ratio.

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2 , shuffle=True , random_state=4)

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

## Baseline Classifer
A baseline classifier will be used to provide a simple, minimalistic model that serves as a reference point for evaluating the performance of more complex models. This establish a performance benchmark when we make our comparison with other models and our final justification in picking the best machine learning technique to generate the final model for our predicition.

In [ ]:
#baseline classifier

dummy_classifier = DummyClassifier(strategy='most_frequent')
dummy_classifier.fit(X_train , y_train)
y_pred = dummy_classifier.predict(X_test)
accuracy = accuracy_score(y_test , y_pred)
print(f"Baseline Model Accuracy: {accuracy:.4f}")

## Logistic Regression
A regression analysis describe data and explain relationship between 1 binary variable and one or more nominal, ordinal or interval independent variables. The outcome from this analysis will give a 'Yes' or 'No' result.

In [ ]:
#logistic regression model

lr_model = make_pipeline(SimpleImputer(strategy='mean') , MinMaxScaler() , LogisticRegression(penalty='l2' , C= 12 ,max_iter=1500))
lr_model.fit(X_train , y_train)

In [ ]:
lr_model.score(X_train , y_train)

In [ ]:
lr_pred = lr_model.predict(X_test)
lr_acc_score = accuracy_score(y_test , lr_pred)
lr_acc_score

In [ ]:
# Coefficients Plot
coefficients = lr_model.named_steps['logisticregression'].coef_.flatten()
feature_names = X_train.columns
plt.figure(figsize=(10, 6))
plt.barh(feature_names, coefficients)
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Coefficients of Logistic Regression Model')
plt.show()

# ROC Curve
y_pred_proba = lr_model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve (AUC = {:.2f})'.format(roc_auc_score(y_test, y_pred_proba)))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

# Precision-Recall Curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label='Precision-Recall Curve (AP = {:.2f})'.format(average_precision_score(y_test, y_pred_proba)))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()

# Confusion Matrix
confusion_matrix_display = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test, lr_model.predict(X_test)), display_labels=["Class 0", "Class 1"])
confusion_matrix_display.plot()
plt.title('Confusion Matrix')
plt.show()

# Learning Curve
train_sizes, train_scores, test_scores = learning_curve(lr_model, X_train, y_train, cv=5)
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training Score')
plt.plot(train_sizes, np.mean(test_scores, axis=1), label='Validation Score')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc="best")
plt.show()

In [ ]:
#Print out FPR TPR

## Decision Tree

In [ ]:
#decision tree model

dt_model = make_pipeline(SimpleImputer(strategy='mean') , MinMaxScaler() , DecisionTreeClassifier())
dt_model.fit(X_train , y_train)


In [ ]:
dt_pred = dt_model.predict(X_test)
dt_acc_score = accuracy_score(y_test , dt_pred)
dt_acc_score

In [ ]:
ConfusionMatrixDisplay.from_estimator(dt_model , X_test , y_test);

In [ ]:
#Print out FPR TPR

## Random Forest

In [66]:
#random forest model

rf_model = make_pipeline(SimpleImputer(strategy='mean') , MinMaxScaler() , RandomForestClassifier(n_estimators=500))
rf_model.fit(X_train , y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('minmaxscaler', MinMaxScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=500))])

In [67]:
y_pred_train = rf_model.predict(X_train)
y_prob_train = rf_model.predict_proba(X_train)[:,1]

y_pred = rf_model.predict(X_test)
y_prob = rf_model.predict_proba(X_test)[:,1]

print('Classification report for test:\n',classification_report(y_test,y_pred))

Classification report for test:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94       639
           1       0.00      0.00      0.00        85

    accuracy                           0.88       724
   macro avg       0.44      0.50      0.47       724
weighted avg       0.78      0.88      0.83       724



In [68]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

rfc = RandomForestClassifier(random_state=1)

params = {
    'n_estimators': sp_randint(5, 25),
    'criterion': ['gini', 'entropy'],
    'max_depth': sp_randint(2, 10),
    'min_samples_split': sp_randint(2, 20),
    'min_samples_leaf': sp_randint(1, 20),
    'max_features': sp_randint(2, 15)
}

rand_search_rfc = RandomizedSearchCV(rfc, param_distributions=params, cv=3, random_state=1)

rand_search_rfc.fit(X, y)
print(rand_search_rfc.best_params_)

{'criterion': 'entropy', 'max_depth': 9, 'max_features': 2, 'min_samples_leaf': 17, 'min_samples_split': 3, 'n_estimators': 17}


In [69]:
rfc = RandomForestClassifier(**rand_search_rfc.best_params_)

rfc.fit(X_train, y_train)

y_pred_train = rfc.predict(X_train)
y_prob_train = rfc.predict_proba(X_train)[:, 1]

y_pred = rfc.predict(X_test)
y_prob = rfc.predict_proba(X_test)[:, 1]

print('Classification report for test:\n', classification_report(y_test, y_pred))

Classification report for test:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94       639
           1       0.00      0.00      0.00        85

    accuracy                           0.88       724
   macro avg       0.44      0.50      0.47       724
weighted avg       0.78      0.88      0.83       724



C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
